In [4]:
import requests
import time
import execjs
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import math
from multiprocessing import Process


In [5]:
pd.set_option('expand_frame_repr', False)
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',1000)

In [6]:
def getAllCode():
    url = 'http://fund.eastmoney.com/js/fundcode_search.js'
    content = requests.get(url)
    jsContent = execjs.compile(content.text)
    rawData = jsContent.eval('r')
    allCode = []
    for code in rawData:
        allCode.append(code[0])
    return allCode


In [7]:
# 文件加载和处理
# fund_narrow_pd=pd.read_csv('./narrow/fund_narrow_pd.csv',low_memory=False, header = None)
# worth_pd文件 最长是 202001
# rateInSimilar_pd最长是040001
# grandTotal_pd 最长是 005503
# 470060
grandTotal_pd=pd.read_csv('./grandTotal/grandTotal_pd.csv',low_memory=False, header = None)
# 040001
rateInSimilar_pd=pd.read_csv('./rateInSimilar/rateInSimilar_pd.csv',low_memory=False, header = None)
# 202001
worth_pd = pd.read_csv(open('./worth/worth_pd.csv','r'),low_memory=False, header = None)
fund_wide_pd=pd.read_csv('./wide/fund_wide_pd.csv',low_memory=False)

worth_pd = worth_pd.fillna(0)
grandTotal_pd = grandTotal_pd.fillna(0)
rateInSimilar_pd = rateInSimilar_pd.fillna(0)

print(grandTotal_pd.info())
print(rateInSimilar_pd.info())
print(worth_pd.info())
print(fund_wide_pd.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25780 entries, 0 to 25779
Columns: 134 entries, 0 to 133
dtypes: float64(132), int64(1), object(1)
memory usage: 26.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26692 entries, 0 to 26691
Columns: 4306 entries, 0 to 4305
dtypes: float64(4304), int64(1), object(1)
memory usage: 876.9+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33365 entries, 0 to 33364
Columns: 4348 entries, 0 to 4347
dtypes: float64(4346), int64(1), object(1)
memory usage: 1.1+ GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6673 entries, 0 to 6672
Data columns (total 32 columns):
name           6673 non-null object
code           6673 non-null int64
类型             6673 non-null object
原费率            5198 non-null float64
现费率            5192 non-null float64
最小申购金额         5194 non-null float64
近一年收益率         5899 non-null float64
近6月收益率         6130 non-null float64
近三月收益率         6218 non-null float64
近一月收益率         6370 non-nul

In [22]:
# 近 x 年来的均值
def get_recent_year_mean(code, colName, target_pd,isChange = False):
    fund_data = target_pd.loc[(target_pd[0] == colName)
                              & (target_pd[1] == code)]
    if fund_data.shape[0] > 0:
        # 排除前两列
        fund_data = fund_data.iloc[:, 2:]
        fund_data = fund_data.T
        fund_data = fund_data[fund_data.iloc[:, 0] > 0]
        day = fund_data.shape[0]
        #     获取年份，向下取整
        yd = 252
        year = math.floor(day / yd)
        for_year = [5 * yd, 4 * yd, 3 * yd, 2 * yd, 1 * yd, 22*6, 22*3, 22]
        year_mean = []

        year_mean_pd = pd.DataFrame([[0,0,0,0,0,0,0,0]],columns=[
            colName + '5年平均值',
            colName + '4年平均值',
            colName + '3年平均值',
            colName + '2年平均值',
            colName + '1年平均值',
            colName + '180天平均值',
            colName + '90天平均值',
            colName + '30天平均值'
        ])
        #     循环年份，不足的标记0
        i = 0;
        for y in for_year:
            fund_data = fund_data.iloc[0:round(y), :]
            mean = fund_data.mean(0)  # 求平均
            if fund_data.shape[0] == y:
                year_mean_pd.iloc[:,i] = mean.values[0]
    #         else:
    #             year_mean_pd.iloc[:,i] = 0
            i += 1
    #     print(year_mean)
    #     print(year_mean_pd)
        return year_mean_pd
    else:
        return None

In [23]:
# 添加worth 和 rateInSimilar 表集合的均值 
fund_wide_avg_pd=pd.read_csv('./wide/fund_wide_pd.csv',low_memory=False)
allCode = getAllCode()
# print(allCode[0],type(allCode[0]))
# allCode = ['1'.zfill(6),'40001'.zfill(6),'3150'.zfill(6)]


pd_list = [worth_pd,rateInSimilar_pd,rateInSimilar_pd,rateInSimilar_pd]
colname_list = ['净值回报','同类排名数','sc','同类排名百分比']
# pd_list = [worth_pd,worth_pd]
# colname_list = ['净值回报','每份派送金']
for i in range(len(pd_list)):
    mean_pd = pd.DataFrame()
    for code in allCode:
        m_pd = get_recent_year_mean(int(code),colname_list[i],pd_list[i])
        if m_pd is not None:
            m_pd.insert(0,'code',int(code))
            mean_pd = mean_pd.append(m_pd, ignore_index=True)
    # 合并均值和原始 wide 并保存
    print("fund_wide_pd",fund_wide_pd.shape)
#     print("mean_pd",mean_pd.info())
#     print("fund_wide_pd",fund_wide_pd.info())
    fund_wide_avg_pd = pd.merge(fund_wide_avg_pd,mean_pd,how='left',on = 'code')
    print("fund_wide_pd",fund_wide_pd.shape)
    print("mean_pd",mean_pd.shape)
    print('完成第',i,'步')
fund_wide_avg_pd.to_csv("./data/fund/fund_wide_avg_1_pd.csv", encoding='utf_8_sig', index=False)
fund_wide_avg_pd=pd.read_csv('./data/fund/fund_wide_avg_1_pd.csv',low_memory=False)
# get_recent_year_mean(4,'净值回报',worth_pd)


fund_wide_pd (6673, 32)
fund_wide_pd (6673, 32)
mean_pd (6666, 9)
完成第 0 步
fund_wide_pd (6673, 32)
fund_wide_pd (6673, 32)
mean_pd (6666, 9)
完成第 1 步
fund_wide_pd (6673, 32)
fund_wide_pd (6673, 32)
mean_pd (6666, 9)
完成第 2 步
fund_wide_pd (6673, 32)
fund_wide_pd (6673, 32)
mean_pd (6666, 9)
完成第 3 步


In [24]:
# 近 x  月来的均值
def get_recent_month_mean(code, colName, target_pd,isChange = False):
    fund_data = target_pd.loc[(target_pd[0] == colName)
                              & (target_pd[1] == code)]
    if fund_data.shape[0] > 0:
        # 排除前两列
        fund_data = fund_data.iloc[:, 2:]
        fund_data = fund_data.T
        fund_data = fund_data[fund_data.iloc[:, 0] > 0]
        day = fund_data.shape[0]
        #     获取年份，向下取整
        year = math.floor(day / 365)
        for_year = [22*6,22*3, 30]
        year_mean = []
        if isChange:
            year_mean_pd = pd.DataFrame([[0,0,0]],columns=[
                'GT各自类型累计收益率' + '180天平均值',
                'GT各自类型累计收益率' + '90天平均值',
                'GT各自类型累计收益率' + '30天平均值'
            ])
        else:
            year_mean_pd = pd.DataFrame([[0,0,0]],columns=[
                colName + '180天平均值',
                colName + '90天平均值',
                colName + '30天平均值'
            ])
        #     循环年份，不足的标记0
        i = 0;
        for y in for_year:
            fund_data = fund_data.iloc[0:round(y), :]
            mean = fund_data.mean(0)  # 求平均
            if fund_data.shape[0] == y:
                year_mean_pd.iloc[:,i] = mean.values[0]
    #         else:
    #             year_mean_pd.iloc[:,i] = 0
            i += 1
    #     print(year_mean)
    #     print(year_mean_pd)
        return year_mean_pd
    else:
        return None

In [25]:
# 添加 gt 表集合的均值 
fund_wide_avg_pd=pd.read_csv('./data/fund/fund_wide_avg_1_pd.csv',low_memory=False)
allCode = getAllCode()
# print(allCode[0],type(allCode[0]))
# allCode = ['1'.zfill(6),'40001'.zfill(6),'3150'.zfill(6)]


pd_list = [grandTotal_pd,grandTotal_pd]
colname_list = ['沪深300累计收益率','同类平均累计收益率']
# pd_list = [worth_pd,worth_pd]
# colname_list = ['净值回报','每份派送金']
for i in range(len(pd_list)):
    mean_pd = pd.DataFrame()
    for code in allCode:
        m_pd = get_recent_month_mean(int(code),colname_list[i],pd_list[i])
        if m_pd is not None:
            m_pd.insert(0,'code',int(code))
            mean_pd = mean_pd.append(m_pd, ignore_index=True)
    # 合并均值和原始 wide 并保存
    print("fund_wide_avg_pd",fund_wide_avg_pd.shape)
#     print("mean_pd",mean_pd.info())
#     print("fund_wide_pd",fund_wide_pd.info())
    fund_wide_avg_pd = pd.merge(fund_wide_avg_pd,mean_pd,how='left',on = 'code')
    print("fund_wide_avg_pd",fund_wide_avg_pd.shape)
    print("mean_pd",mean_pd.shape)
    print('完成第',i,'步')
fund_wide_avg_pd.to_csv("./data/fund/fund_wide_avg_2_pd.csv", encoding='utf_8_sig', index=False)
fund_wide_avg_pd=pd.read_csv('./data/fund/fund_wide_avg_2_pd.csv',low_memory=False)
# get_recent_year_mean(4,'净值回报',worth_pd)


fund_wide_pd (6673, 32)
fund_wide_pd (6673, 32)
mean_pd (6495, 4)
完成第 0 步
fund_wide_pd (6673, 32)
fund_wide_pd (6673, 32)
mean_pd (6271, 4)
完成第 1 步


In [26]:
# 计算grandTotal_pd最后行不定名称
fund_wide_avg_gt_pd=pd.read_csv('./data/fund/fund_wide_avg_2_pd.csv',low_memory=False)

# print(fund_data.shape)
# print(type(name))
allCode = getAllCode()
# allCode = ['040001','003150']
mean_pd = pd.DataFrame()
for code in allCode:
    fund_data = grandTotal_pd.loc[grandTotal_pd[1] == int(code)]
    if fund_data.shape[0] > 0:
        name = fund_data.iloc[fund_data.shape[0] - 1,0]
        m_pd = get_recent_month_mean(int(code),name,grandTotal_pd,True)
        if m_pd is not None:
            m_pd.insert(0,'code',int(code))
            mean_pd = mean_pd.append(m_pd, ignore_index=True)
# 合并均值和原始 wide 并保存
print("fund_wide_pd",fund_wide_pd.shape)
print("mean_pd",mean_pd.shape)
# print("mean_pd",mean_pd)
fund_wide_avg_gt_pd = pd.merge(fund_wide_avg_gt_pd,mean_pd,how='left',on = 'code')
print("fund_wide_pd",fund_wide_pd.shape)
fund_wide_avg_gt_pd.to_csv("./data/fund/fund_wide_avg_gt_pd.csv", encoding='utf_8_sig', index=False)
fund_wide_avg_gt_pd=pd.read_csv('./data/fund/fund_wide_avg_gt_pd.csv',low_memory=False)


fund_wide_pd (6673, 32)
mean_pd (6495, 4)
fund_wide_pd (6673, 32)


In [30]:
# 处理基金经理
# 统计指标，成立以来有过 N 名经理，最近更换经理到现在的天数
def get_recent_manage_info(code):
    manage_single_pd = pd.DataFrame(columns=[
        '经理ID', ' 经理星级', '经理在这个基金工作时间', '经理资金数', '经理基金数', '经理平均分', '经理经验值',
        '经理收益率', '经理抗风险', '经理稳定性', '经理择时能力', '经理任期收益', '经理同类平均', '经理沪深300',
        '经理跟踪误差', '经理超额收益', '经理管理规模'
    ])
    manages = fund_wide_pd.loc[fund_wide_pd['code'] == int(code)]['经理']
    if manages.shape[0] > 0:
        manage_list = fund_wide_pd.loc[fund_wide_pd['code'] ==
                                            int(code)]['经理'].values[0].split('\n')
        manage_pd = pd.DataFrame()
        manage_c = len(manage_list[0].split())  # 经理人数

        manage_list[0] = '列名 ' + manage_list[0]
        # 把经理恢复成 pd，可操作
        for row in manage_list:
            row_s = row.split()
            series = pd.Series(row_s[1:], name=row_s[0]).T
            manage_pd = manage_pd.append(series)
#         print(manage_pd)

#         print(manage_pd.shape)
#         print(manage_pd)
        if manage_pd.shape[0] == 3:
            print("空经理 code", code)
            return manage_single_pd
        manage_wordDay_Total = 0
        manage_id = '0'
        target = -1
        #收益率
        #     找出时间最长的经理
        #     workDay_s = manage_pd.loc[manage_pd[0] == '工作时间']
        #     print("manage_c",manage_c)
        for i in range(manage_c):
            #             try:
            workDay_s = manage_pd.loc[['工作时间'], i]
            #             except:
            #                 return []
            manage_wordDay = workDay_s[0].split('又')
            #     起码1 年以上经历
            if len(manage_wordDay) > 1:
                manage_word_year = manage_wordDay[0].split('年')[0]
                manage_word_day = manage_wordDay[1].split('天')[0]
                manage_wordDay_Total_new = int(manage_word_year) * 365 + int(
                    manage_word_day)
                if manage_wordDay_Total_new > manage_wordDay_Total:
                    manage_wordDay_Total = manage_wordDay_Total_new
                    target = i

    #     根据最长经理所在列取数据

        if target != -1:
            target_s = manage_pd.loc[:, target]
            manage_id = target_s['id']
            manage_star = target_s['星级']
            if str(target_s['资金/基金数']) != 'nan':
                manage_fund_money_total = target_s['资金/基金数'].split('亿')[
                    0]  #管理总基金额
                manage_funds_total = target_s['资金/基金数'].split('(')[1].split(
                    '只')[0]  # 管理总基金数量
            else:
                manage_fund_money_total = 0
                manage_funds_total = 0
            manage_point_avg = target_s['平均分']
            manage_exp = target_s['(经验值,)']
            manage_earn_per = target_s['(收益率,)']
            try:
                manage_resist_risk = target_s['(抗风险,)']
            except:
                manage_resist_risk = 0

            try:
                manage_stable = target_s['(稳定性,)']
            except:
                manage_stable = 0

            try:
                manage_choice = target_s['(择时能力,)']
            except:
                manage_choice = 0

            try:
                manage_eran_term = target_s['(任期收益,)']
            except:
                manage_eran_term = 0

            try:
                manage_similar_avg = target_s['(同类平均,)']
            except:
                manage_similar_avg = 0

            try:
                manage_hs300 = target_s['(沪深300,)']
            except:
                manage_hs300 = 0

            try:
                manage_track_error = target_s['(跟踪误差,)']
            except:
                manage_track_error = 0

            try:
                manage_eran_excess = target_s['(超额收益,)']
            except:
                manage_eran_excess = 0

            try:
                manage_manage_scale = target_s['(管理规模,)']
            except:
                manage_manage_scale = 0

            insertRow = pd.DataFrame(
                [[
                    manage_id, manage_star, manage_wordDay_Total,
                    manage_fund_money_total, manage_funds_total,
                    manage_point_avg, manage_exp, manage_earn_per,
                    manage_resist_risk, manage_stable, manage_choice,
                    manage_eran_term, manage_similar_avg, manage_hs300,
                    manage_track_error, manage_eran_excess, manage_manage_scale
                ]],
                columns=[
                    '经理ID', ' 经理星级', '经理在这个基金工作时间', '经理资金数', '经理基金数', '经理平均分',
                    '经理经验值', '经理收益率', '经理抗风险', '经理稳定性', '经理择时能力', '经理任期收益',
                    '经理同类平均', '经理沪深300', '经理跟踪误差', '经理超额收益', '经理管理规模'
                ])
            manage_single_pd = manage_single_pd.append(
                insertRow, ignore_index=True)
            return manage_single_pd
        else:
#             print("target=-1,code=",code)
            return manage_single_pd
    else:
#         print("manages.shape[0]=0,code=",code)
        return manage_single_pd

fund_wide_manage_pd=pd.read_csv('./data/fund/fund_wide_avg_gt_pd.csv',low_memory=False)
allCode = getAllCode()
# allCode = ['040001','003150']
manages_pd = pd.DataFrame()
for code in allCode:
    manage_s_pd = get_recent_manage_info(code)
    manage_s_pd.insert(0,'code',int(code))
    manages_pd = manages_pd.append(manage_s_pd, ignore_index=True)
    
# 合并经理和原始 wide 并保存
print(fund_wide_pd.shape)
fund_wide_manage_pd = pd.merge(fund_wide_manage_pd,manages_pd,how='left',on = 'code')
print(fund_wide_pd.shape)
print(manages_pd.shape)
fund_wide_manage_pd.to_csv("./data/fund/fund_wide_manage_pd.csv", encoding='utf_8_sig', index=False)
fund_wide_manage_pd=pd.read_csv('./data/fund/fund_wide_manage_pd.csv',low_memory=False)

空经理 code 001729
(6673, 32)
(6673, 32)
(6375, 18)


In [31]:
# feature_data_pd=pd.read_csv('./data/fund/feature_data_pd.csv',low_memory=False)
fund_wide_manage_pd=pd.read_csv('./data/fund/fund_wide_manage_pd.csv',low_memory=False)
fund_wide_avg_gt_pd=pd.read_csv('./data/fund/fund_wide_avg_gt_pd.csv',low_memory=False)
# print(feature_data_pd.info())
print(fund_wide_manage_pd.info())
print(fund_wide_avg_gt_pd.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6673 entries, 0 to 6672
Data columns (total 90 columns):
name                  6673 non-null object
code                  6673 non-null int64
类型                    6673 non-null object
原费率                   5198 non-null float64
现费率                   5192 non-null float64
最小申购金额                5194 non-null float64
近一年收益率                5899 non-null float64
近6月收益率                6130 non-null float64
近三月收益率                6218 non-null float64
近一月收益率                6370 non-null float64
基金持仓股票代码              6672 non-null object
基金持仓债券代码              6673 non-null object
规模变动日期                6673 non-null object
规模变动净资产亿元             6673 non-null object
规模变动较上期环比             6673 non-null object
('机构持有比例',)           6673 non-null object
('个人持有比例',)           6673 non-null object
('内部持有比例',)           6673 non-null object
('股票占净比',)            6673 non-null object
('债券占净比',)            6673 non-null object
('现金占净比',)            6673 

In [32]:
# 合并特色数据
feature_data_pd=pd.read_csv('./data/fund/feature_data_pd.csv',low_memory=False)
fund_wide_manage_pd=pd.read_csv('./data/fund/fund_wide_manage_pd.csv',low_memory=False)

feature_data_pd = pd.merge(fund_wide_manage_pd,feature_data_pd,how='left',on='code')
feature_data_pd.to_csv("./data/fund/fund_wide_feature_data_pd.csv", encoding='utf_8_sig', index=False)

In [33]:
# 合并综合评级
fund_wide_feature_data_pd=pd.read_csv('./data/fund/fund_wide_feature_data_pd.csv',low_memory=False)
fund_rating_pd=pd.read_csv('./data/fund/fund_rating_pd.csv',low_memory=False)
fund_rating_pd = pd.merge(fund_wide_feature_data_pd,fund_rating_pd,how='left',on='code')
fund_rating_pd.to_csv("./data/fund/fund_wide_rating_data_pd.csv", encoding='utf_8_sig', index=False)

In [34]:
# 合并3家评级
fund_rating_pd=pd.read_csv('./data/fund/fund_wide_rating_data_pd.csv',low_memory=False)
fund_shzq_rating_pd=pd.read_csv('./data/fund/fund_shzq_rating_pd.csv',low_memory=False)
fund_zszq_rating_pd=pd.read_csv('./data/fund/fund_zszq_rating_pd.csv',low_memory=False)
fund_jajx_rating_pd=pd.read_csv('./data/fund/fund_jajx_rating_pd.csv',low_memory=False)

fund_rating_pd = pd.merge(fund_rating_pd,fund_shzq_rating_pd,how='left',on='code')
fund_rating_pd = pd.merge(fund_rating_pd,fund_zszq_rating_pd,how='left',on='code')
fund_rating_pd = pd.merge(fund_rating_pd,fund_jajx_rating_pd,how='left',on='code')

fund_rating_pd.to_csv("./data/fund/fund_wide_rating_1_data_pd.csv", encoding='utf_8_sig', index=False)

In [120]:
# 分红计算周期内次数和总金额
def getDividend(code,target_pd):
#     print('code：',code)
    colName = '分红'
    fund_data = target_pd.loc[(target_pd[0] == '每份派送金')
                              & (target_pd[1] == code)]
    if fund_data.shape[0] > 0:
        # 排除前两列
        fund_data = fund_data.iloc[:, 2:]
        fund_data = fund_data.T
        fund_data = fund_data[fund_data.iloc[:, 0] > 0]
        day = fund_data.shape[0]
        #     获取年份，向下取整
        yd = 252
        year = math.floor(day / yd)
        for_year = [5 * yd, 4 * yd, 3 * yd, 2 * yd, 1 * yd, 22*6, 22*3, 22]

        year_count_pd = pd.DataFrame([[0,0,0,0,0,0,0,0]],columns=[
            colName + '5年次数',
            colName + '4年次数',
            colName + '3年次数',
            colName + '2年次数',
            colName + '1年次数',
            colName + '180天次数',
            colName + '90天次数',
            colName + '30天次数'
        ])
        
        year_amount_pd = pd.DataFrame([[0,0,0,0,0,0,0,0]],columns=[
            colName + '5年总金额',
            colName + '4年总金额',
            colName + '3年总金额',
            colName + '2年总金额',
            colName + '1年总金额',
            colName + '180天总金额',
            colName + '90天总金额',
            colName + '30天总金额'
        ])
        #     循环年份，不足的标记0
        i = 0;
        for y in for_year:
            if fund_data.shape[0] >= y:
                fund_data = fund_data.iloc[0:round(y), :]
                count = fund_data.where( fund_data > 0).shape[0]  # 求次数
                amount = fund_data.sum()
#                 if fund_data.shape[0] is not None:
    #                 print('amount',type(amount.iloc[0]))
                year_count_pd.iloc[:,i] = count

                year_amount_pd.iloc[:,i] = amount.iloc[0]
            i += 1
#         print(year_amount_pd)
        dividend_pd = pd.concat([year_count_pd, year_amount_pd], axis=1)
        return dividend_pd
    else:
        return None
    
# 开始计算分红数据
fund_wide_dividend_pd = pd.read_csv('./data/fund/fund_wide_rating_1_data_pd.csv',low_memory=False)
# getDividend(1,worth_pd)

dividend_pd = pd.DataFrame()
# print(fund_data.shape)
# print(type(name))
allCode = getAllCode()
# allCode = ['005208','001264']
# mean_pd = pd.DataFrame()
for code in allCode:
#     fund_data = grandTotal_pd.loc[grandTotal_pd[1] == int(code)]
#     if fund_data.shape[0] > 0:
#     name = fund_data.iloc[fund_data.shape[0] - 1,0]
    r_pd = getDividend(int(code),worth_pd)
    if r_pd is not None:
        r_pd.insert(0,'code',int(code))
        dividend_pd = dividend_pd.append(r_pd, ignore_index=True)
# 合并均值和原始 wide 并保存
print("fund_wide_dividend_pd",fund_wide_dividend_pd.shape)
print("dividend_pd",dividend_pd.shape)
# print(dividend_pd)
fund_wide_dividend_pd = pd.merge(fund_wide_dividend_pd,dividend_pd,how='left',on = 'code')
print("fund_wide_dividend_pd",fund_wide_dividend_pd.shape)
fund_wide_dividend_pd.to_csv("./data/fund/fund_wide_dividend_pd.csv", encoding='utf_8_sig', index=False)
fund_wide_dividend_pd=pd.read_csv('./data/fund/fund_wide_dividend_pd.csv',low_memory=False)

fund_wide_dividend_pd (6673, 116)
dividend_pd (6666, 17)
fund_wide_dividend_pd (6673, 132)


In [167]:
# 净值回报 计算周期内根据首尾天单位净值的计算
def getNetWorthReport(code, target_pd):
    #     print('code：',code)
    colName = '净值回报率'
    fund_data = target_pd.loc[(target_pd[0] == '单位净值')
                              & (target_pd[1] == code)]
    if fund_data.shape[0] > 0:
        # 排除前两列
        fund_data = fund_data.iloc[:, 2:]
        fund_data = fund_data.T
        fund_data = fund_data[fund_data.iloc[:, 0] > 0]
        day = fund_data.shape[0]
        #     获取年份，向下取整
        yd = 252
        year = math.floor(day / yd)
        for_year = [5 * yd, 4 * yd, 3 * yd, 2 * yd, 1 * yd, 22 * 6, 22 * 3, 22]

        year_report_pd = pd.DataFrame(
            [[0, 0, 0, 0, 0, 0, 0, 0]],
            columns=[
                '近5年' + colName, '近4年' + colName, '近3年' + colName,
                '近2年' + colName, '近1年' + colName, '近180天' + colName,
                '近90天' + colName, '近30天' + colName
            ])
        #     循环年份，不足的标记0
        i = 0
        for y in for_year:
            if fund_data.shape[0] >= y:
                fund_data = fund_data.iloc[0:round(y), :]

                
                # 求净值回报率
                report = (fund_data.iloc[0,:] - fund_data.iloc[-1,:])/fund_data.iloc[-1,:]*100
                if report.iloc[0] is not None:
                    year_report_pd.iloc[:, i] = report.iloc[0]
            i += 1
        return year_report_pd
    else:
        return None

In [169]:
# 开始计算回报率
fund_wide_report_pd = pd.read_csv('./data/fund/fund_wide_dividend_pd.csv',low_memory=False)

report_pd = pd.DataFrame()
allCode = getAllCode()
# allCode = ['005208','001264']
for code in allCode:
    r_pd = getNetWorthReport(int(code),worth_pd)
    if r_pd is not None:
        r_pd.insert(0,'code',int(code))
        report_pd = report_pd.append(r_pd, ignore_index=True)
# 合并均值和原始 wide 并保存
print("fund_wide_report_pd",fund_wide_report_pd.shape)
print("report_pd",report_pd.shape)
# print(report_pd)
fund_wide_report_pd = pd.merge(fund_wide_report_pd,report_pd,how='left',on = 'code')
print("fund_wide_report_pd",fund_wide_report_pd.shape)
fund_wide_report_pd.to_csv("./data/fund/fund_wide_report_pd.csv", encoding='utf_8_sig', index=False)
fund_wide_report_pd=pd.read_csv('./data/fund/fund_wide_report_pd.csv',low_memory=False)

fund_wide_report_pd (6673, 132)
report_pd (6666, 9)
        code    近5年净值回报率   近4年净值回报率   近3年净值回报率   近2年净值回报率   近1年净值回报率  近180天净值回报率  近90天净值回报率  近30天净值回报率
0          3  -21.560575 -29.650092 -13.476784  -7.729469  12.850812   -0.391134  13.690476   6.111111
1          4  -21.052632 -30.264357 -14.141414  -8.052885  12.831858   -0.260756  13.670134   6.102635
2          5    2.362205   0.096246   0.192678   1.860921   0.192678   -1.140684   0.677638   0.289296
3          6    0.000000   0.000000   0.000000   0.000000   0.000000    0.000000  16.309462   7.353083
4          8   16.404287 -33.402003 -20.228394 -21.418881   2.955118   -4.545767   7.576227   8.474444
5         11   78.799730  16.584594  26.098535   9.624607  14.224175    5.022180  -0.525210   3.651005
6         14   21.047431   9.865471   3.550296   4.700855   8.695652   -0.809717   7.174103   2.253756
7         15   20.837390  10.409253   5.258694   6.615120   1.388889    2.816901   1.721311   0.080645
8         16   18.217

In [171]:
# GT 表 周期内首-尾天排名差
# 净值回报 计算周期内根据首尾天单位净值的计算
def getRatingDiff(code, target_pd, colName):
    #     print('code：',code)
    fund_data = target_pd.loc[(target_pd[0] == colName)
                              & (target_pd[1] == code)]
    if fund_data.shape[0] > 0:
        # 排除前两列
        fund_data = fund_data.iloc[:, 2:]
        fund_data = fund_data.T
        fund_data = fund_data[fund_data.iloc[:, 0] > 0]
        day = fund_data.shape[0]
        #     获取年份，向下取整
        yd = 252
        year = math.floor(day / yd)
        for_year = [5 * yd, 4 * yd, 3 * yd, 2 * yd, 1 * yd, 22 * 6, 22 * 3, 22]

        year_similar_diff_pd = pd.DataFrame([[0,0,0,0,0,0,0,0]],columns=[
            colName + '5年上升',
            colName + '4年上升',
            colName + '3年上升',
            colName + '2年上升',
            colName + '1年上升',
            colName + '180天上升',
            colName + '90天上升',
            colName + '30天上升'
        ])

        #     循环年份，不足的标记0
        i = 0
        for y in for_year:
            if fund_data.shape[0] >= y:
                fund_data = fund_data.iloc[0:round(y), :]
                # 求排名差
                diff = fund_data.iloc[0,:] - fund_data.iloc[-1,:]
                if diff.iloc[0] is not None:
                    year_similar_diff_pd.iloc[:, i] = diff.iloc[0]
            i += 1
        return year_similar_diff_pd
    else:
        return None

In [184]:
# 开始计算排名差
fund_wide_rating_diff_pd = pd.read_csv('./data/fund/fund_wide_report_pd.csv',low_memory=False)


allCode = getAllCode()
# allCode = ['005208','001264']
# allCode = ['005208']
pd_list = [rateInSimilar_pd,rateInSimilar_pd]
colname_list = ['同类排名数','sc']
for i in range(len(pd_list)):
    rating_diff_pd = pd.DataFrame()
    for code in allCode:
        r_pd = getRatingDiff(int(code),pd_list[i],colname_list[i])
#         print("rpd",r_pd)
        if r_pd is not None:
            r_pd.insert(0,'code',int(code))
            rating_diff_pd = rating_diff_pd.append(r_pd, ignore_index=True)
#             print("rating_diff_pd",rating_diff_pd)
    # 合并均值和原始 wide 并保存
    print("fund_wide_rating_diff_pd",fund_wide_rating_diff_pd.shape)
    print("rating_diff_pd",rating_diff_pd.shape)
    # print(rating_diff_pd)
    fund_wide_rating_diff_pd = pd.merge(fund_wide_rating_diff_pd,rating_diff_pd,how='left',on = 'code')
print("fund_wide_rating_diff_pd",fund_wide_rating_diff_pd.shape)
fund_wide_rating_diff_pd.to_csv("./data/fund/fund_wide_rating_diff_pd.csv", encoding='utf_8_sig', index=False)
fund_wide_rating_diff_pd=pd.read_csv('./data/fund/fund_wide_rating_diff_pd.csv',low_memory=False)

fund_wide_rating_diff_pd (6673, 140)
rating_diff_pd (6666, 9)
fund_wide_rating_diff_pd (6673, 148)
rating_diff_pd (6666, 9)
fund_wide_rating_diff_pd (6673, 156)


In [264]:
# '沪深300累计收益率','同类平均累计收益率','GT各自累计收益率' 计算周期内各自-同类，各自-沪深的
# 文件是半年的
# 找沪深 300 最多天数的是 400001

fund_wide_rating_diff_pd=pd.read_csv('./data/fund/fund_wide_rating_diff_pd.csv',low_memory=False)
# 需要找各类型最多天数的code

fund_wide_rating_diff_pd['基金成立日期'] = pd.to_datetime(fund_wide_rating_diff_pd['基金成立日期'])
# fund_type_max_create_day = fund_wide_rating_diff_pd.groupby('类型')
# print(fund_type_max_create_day['基金成立日期'].min().shape)
# 找出各类型最早成立的 code
type_min_create_date_code =  fund_wide_rating_diff_pd.groupby('类型').apply(lambda t: t.min()).loc[:,['code','基金成立日期']]


In [275]:
# 处理 object 类型到数字
def fundType2Num(type_name):
    if type_name == 'ETF-场内':
        return 1
    elif type_name == '债券型':
        return 2
    elif type_name == '债券指数':
        return 3
    elif type_name == '其他创新':
        return 4
    elif type_name == '分级杠杆':
        return 5
    elif type_name == '固定收益':
        return 6
    elif type_name == '定开债券':
        return 7
    elif type_name == '混合-FOF':
        return 8
    elif type_name == '混合型':
        return 9
    elif type_name == '理财型':
        return 10
    elif type_name == '联接基金':
        return 11
    elif type_name == '股票型':
        return 12
    elif type_name == '股票指数':
        return 13
    elif type_name == '货币型':
        return 14

def fundInvestStyle(style_name):
    if style_name == '大盘价值':
        return 1
    elif style_name == '大盘平衡':
        return 2
    elif style_name == '大盘成长':
        return 3
    elif style_name == '中盘价值':
        return 4
    elif style_name == '中盘平衡':
        return 5
    elif style_name == '中盘成长':
        return 6
    elif style_name == '小盘价值':
        return 7
    elif style_name == '小盘平衡':
        return 8
    elif style_name == '小盘成长':
        return 9

def replaceNone(none_str):
    if none_str == '暂无数据' or none_str == '--' or none_str == 'None':
        return 0


def replaceCreateDate(dateStr):
    if dateStr == '---':
        return ''
    else:
        return dateStr
    
fund_wide_good_dtype_pd=pd.read_csv('./data/fund/fund_wide_rating_diff_pd.csv',low_memory=False)


fund_wide_good_dtype_pd['基金成立日期'] = fund_wide_good_dtype_pd.apply(
    lambda x:replaceCreateDate(x['基金成立日期']), axis=1)
fund_wide_good_dtype_pd['基金成立日期'] = pd.to_datetime(fund_wide_good_dtype_pd['基金成立日期'])
    
fund_wide_good_dtype_pd['类型'] = fund_wide_good_dtype_pd.apply(
    lambda x: fundType2Num(x['类型']), axis=1)


fund_wide_good_dtype_pd['基金投资风格2019年2季度'] = fund_wide_good_dtype_pd.apply(
    lambda x: fundInvestStyle(x['基金投资风格2019年2季度']), axis=1)

fund_wide_good_dtype_pd['基金投资风格2019年1季度'] = fund_wide_good_dtype_pd.apply(
    lambda x: fundInvestStyle(x['基金投资风格2019年1季度']), axis=1)

fund_wide_good_dtype_pd['基金投资风格2018年4季度'] = fund_wide_good_dtype_pd.apply(
    lambda x: fundInvestStyle(x['基金投资风格2018年4季度']), axis=1)

fund_wide_good_dtype_pd['基金投资风格2018年3季度'] = fund_wide_good_dtype_pd.apply(
    lambda x: fundInvestStyle(x['基金投资风格2018年3季度']), axis=1)

fund_wide_good_dtype_pd['基金投资风格2018年2季度'] = fund_wide_good_dtype_pd.apply(
    lambda x: fundInvestStyle(x['基金投资风格2018年2季度']), axis=1)

fund_wide_good_dtype_pd['基金投资风格2018年1季度'] = fund_wide_good_dtype_pd.apply(
    lambda x: fundInvestStyle(x['基金投资风格2018年1季度']), axis=1)

fund_wide_good_dtype_pd['基金投资风格2017年4季度'] = fund_wide_good_dtype_pd.apply(
    lambda x: fundInvestStyle(x['基金投资风格2017年4季度']), axis=1)

fund_wide_good_dtype_pd['基金投资风格2017年3季度'] = fund_wide_good_dtype_pd.apply(
    lambda x: fundInvestStyle(x['基金投资风格2017年3季度']), axis=1)

fund_wide_good_dtype_pd["('平均值',)"] = fund_wide_good_dtype_pd.apply(
    lambda x: replaceNone(x["('平均值',)"]), axis=1)

fund_wide_good_dtype_pd['经理平均分'] = fund_wide_good_dtype_pd.apply(
    lambda x: replaceNone(x['经理平均分']), axis=1)

fund_wide_good_dtype_pd['经理经验值'] = fund_wide_good_dtype_pd.apply(
    lambda x: replaceNone(x['经理经验值']), axis=1)

fund_wide_good_dtype_pd['经理收益率'] = fund_wide_good_dtype_pd.apply(
    lambda x: replaceNone(x['经理收益率']), axis=1)

fund_wide_good_dtype_pd['经理抗风险'] = fund_wide_good_dtype_pd.apply(
    lambda x: replaceNone(x['经理抗风险']), axis=1)

fund_wide_good_dtype_pd['经理稳定性'] = fund_wide_good_dtype_pd.apply(
    lambda x: replaceNone(x['经理稳定性']), axis=1)

fund_wide_good_dtype_pd['经理择时能力'] = fund_wide_good_dtype_pd.apply(
    lambda x: replaceNone(x['经理择时能力']), axis=1)

fund_wide_good_dtype_pd['经理跟踪误差'] = fund_wide_good_dtype_pd.apply(
    lambda x: replaceNone(x['经理跟踪误差']), axis=1)

fund_wide_good_dtype_pd['经理超额收益'] = fund_wide_good_dtype_pd.apply(
    lambda x: replaceNone(x['经理超额收益']), axis=1)

fund_wide_good_dtype_pd['经理管理规模'] = fund_wide_good_dtype_pd.apply(
    lambda x: replaceNone(x['经理管理规模']), axis=1)

fund_wide_good_dtype_pd['标准差近1年'] = fund_wide_good_dtype_pd.apply(
    lambda x: replaceNone(x['标准差近1年']), axis=1)

fund_wide_good_dtype_pd['标准差近2年'] = fund_wide_good_dtype_pd.apply(
    lambda x: replaceNone(x['标准差近2年']), axis=1)

fund_wide_good_dtype_pd['标准差近3年'] = fund_wide_good_dtype_pd.apply(
    lambda x: replaceNone(x['标准差近3年']), axis=1)

fund_wide_good_dtype_pd['夏普比率近1年'] = fund_wide_good_dtype_pd.apply(
    lambda x: replaceNone(x['夏普比率近1年']), axis=1)

fund_wide_good_dtype_pd['夏普比率近2年'] = fund_wide_good_dtype_pd.apply(
    lambda x: replaceNone(x['夏普比率近2年']), axis=1)

fund_wide_good_dtype_pd['夏普比率近3年'] = fund_wide_good_dtype_pd.apply(
    lambda x: replaceNone(x['夏普比率近3年']), axis=1)

fund_wide_good_dtype_pd.to_csv("./data/fund/fund_wide_good_dtype_pd.csv", encoding='utf_8_sig', index=False)
fund_wide_good_dtype_pd=pd.read_csv('./data/fund/fund_wide_good_dtype_pd.csv',low_memory=False)

# for c_name in fund_wide_good_dtype_pd.columns.tolist():
#     if fund_wide_good_dtype_pd[c_name].dtypes == 'object':
#         print(c_name)



name
基金持仓股票代码
基金持仓债券代码
规模变动日期
规模变动净资产亿元
规模变动较上期环比
('机构持有比例',)
('个人持有比例',)
('内部持有比例',)
('股票占净比',)
('债券占净比',)
('现金占净比',)
('净资产',)
经理
('期间申购',)
('期间赎回',)
('总份额',)
company_name
基金成立日期
手续费


In [279]:
for c_name in fund_wide_good_dtype_pd.columns.tolist():
    if fund_wide_good_dtype_pd[c_name].dtypes == 'object':
        print(c_name)


name
基金持仓股票代码
基金持仓债券代码
规模变动日期
规模变动净资产亿元
规模变动较上期环比
('机构持有比例',)
('个人持有比例',)
('内部持有比例',)
('股票占净比',)
('债券占净比',)
('现金占净比',)
('净资产',)
经理
('期间申购',)
('期间赎回',)
('总份额',)
company_name
基金成立日期
手续费


In [ ]:
# 要爬的历史数据，基金档案


# 持有人变动详情 http://fundf10.eastmoney.com/cyrjg_159901.html 
# 资产配置明细 http://fundf10.eastmoney.com/zcpz_159901.html 
# 持仓明细 http://fundf10.eastmoney.com/ccmx_519670.html

In [ ]:
# 阶段涨幅??? 累计净值符合成立以来数据
(1.6030-1.4240)/1.4240*100=12.57
(1.6030-1)/1*100=60.3
(3.1280-3.0270)/3.0270=3.336636934
(3.1280- 1)/1*100=312.8

In [278]:
# 年限应该看经理最后一次更换的时间
# 各数据波动率

In [ ]:
# 淘汰方法，但保留

In [14]:
# 找 N 年以上的基金
def get_year_funds(years, allCode):
    funds_year = []
#     allCode = ['000008']
    for code in allCode:
#         print(code)
        fund_day = rateInSimilar_pd.loc[(rateInSimilar_pd[0] == "时间戳") & (rateInSimilar_pd[1] ==  int(code))]
        if fund_day.shape[0] > 0:
            fund_day = fund_day.T
#             print(fund_day.shape)
#             print(fund_day)
            day_c = fund_day[fund_day.iloc[:,0] != 0].shape[0]
            if day_c >= 365 * years + 10:
#                 print("code",code,"day_c",day_c)
                funds_year.append(code)
    return funds_year

In [15]:
# 找出有 5 年和 3 年的基金
allCode = getAllCode()
funds_5y = get_year_funds(5,allCode)
pd.DataFrame(funds_5y).to_csv("./funds_5y.csv", header=False, index=False)
funds_3y = get_year_funds(3,allCode)
pd.DataFrame(funds_3y).to_csv("./funds_3y.csv", header=False, index=False)

In [16]:
# 近 N 年 同类绩效均值 n(0-1)
# 数值越大越好，表示收益率
def get_similar_per_mean(year):
    if year <= 3:
        file_year = 3
    if year >3 and year <= 5:
        file_year = 5
    fund_similar_mean_list = []
    fund_similar_code_list = []
    fund_year_pd = pd.read_csv(
        './funds_' + str(file_year) + 'y.csv', low_memory=False, header=None)
    fund_year_list = fund_year_pd.values.tolist()
    
#     fund_year_list = [1,11]
    for code in fund_year_list:
#         start1 = time.time()
        code = code[0]
#         print(code)
        fund_similar = rateInSimilar_pd.loc[(rateInSimilar_pd[0] == "同类排名百分比")
                                          & (rateInSimilar_pd[1] == code)]
#         fund_similar.drop([0, 1, 2], axis=1, inplace=True) # 删除前三列无用数据
        fund_similar = fund_similar.T
        fund_similar = fund_similar.iloc[2:,:]
        fund_similar = fund_similar[fund_similar.iloc[:, 0] > 0] # 只要有效数据
        if fund_similar.shape[0] >= year * 365:
            fund_similar = fund_similar.iloc[0: round(year*365) , :]  # 找 x 天内的数据,取前 n 行
#             print(fund_similar.shape)
            mean = fund_similar.mean(0).iloc[0] # 求平均
            fund_similar_code_list.append(code)
            fund_similar_mean_list.append(mean)
            fund_similar_mean_dict = {
                "code" : fund_similar_code_list,
               "同类排名百分比平均值" : fund_similar_mean_list
            } 
#         end1 = time.time()
#         print("for 用时{}".format((end1-start1)))
    fund_similar_mean_pd =  pd.DataFrame(fund_similar_mean_dict)
    
    return fund_similar_mean_pd

def get_similar_per_top(similar_mean,top):
#     排序
    similar_mean.sort_values(by = '同类排名百分比平均值', inplace=True, ascending=False)
#     获取长度
    pd_len = similar_mean.shape[0]
#     取前三分之一
    similar_top = similar_mean.head(round(pd_len * top))
    return similar_top

In [17]:
# 近 N 年 同类绩效均值 n(0-1)
def get_similar_mean(year):
    if year <= 3:
        file_year = 3
    if year >3 and year <= 5:
        file_year = 5
    fund_similar_mean_list = []
    fund_similar_code_list = []
    fund_year_pd = pd.read_csv(
        './funds_' + str(file_year) + 'y.csv', low_memory=False, header=None)
    fund_year_list = fund_year_pd.values.tolist()
    
#     fund_year_list = [1,11]
    for code in fund_year_list:
#         start1 = time.time()
        code = code[0]
#         print(code)
        fund_similar = rateInSimilar_pd.loc[(rateInSimilar_pd[0] == "同类排名数")
                                          & (rateInSimilar_pd[1] == code)]
#         fund_similar.drop([0, 1, 2], axis=1, inplace=True) # 删除前三列无用数据
        fund_similar = fund_similar.T
        fund_similar = fund_similar.iloc[2:,:]
        fund_similar = fund_similar[fund_similar.iloc[:, 0] > 0] # 只要有效数据
        if fund_similar.shape[0] >= year * 365:
            fund_similar = fund_similar.iloc[0: round(year*365) , :]  # 找 x 天内的数据,取前 n 行
#             print(fund_similar.shape)
            mean = fund_similar.mean(0).iloc[0] # 求平均
            fund_similar_code_list.append(code)
            fund_similar_mean_list.append(mean)
            fund_similar_mean_dict = {
                "code" : fund_similar_code_list,
               "同类排名数平均值" : fund_similar_mean_list
            } 
#         end1 = time.time()
#         print("for 用时{}".format((end1-start1)))
    fund_similar_mean_pd =  pd.DataFrame(fund_similar_mean_dict)
    
    return fund_similar_mean_pd

def get_similar_top(similar_mean,top):
#     排序
    similar_mean.sort_values(by = '同类排名数平均值', inplace=True, ascending=True)
#     获取长度
    pd_len = similar_mean.shape[0]
#     取前三分之一
    similar_top = similar_mean.head(round(pd_len * top))
    return similar_top

In [18]:
# 统计5 年来同类排名百分比平均值前 30%的基金
for i in range(1,6,1):
    fund_similar_mean_pd = get_similar_per_mean(i)
    fund_similar_top_pd = get_similar_per_top(fund_similar_mean_pd, 0.3)
    fund_similar_top_pd.to_csv("./fund_similar_top_per_" + str(i) +"y_pd.csv", index=False, encoding='utf_8_sig')
# print(fund_similar_mean_pd)
# 半年
fund_similar_mean_pd = get_similar_per_mean(0.5)
fund_similar_top_pd = get_similar_per_top(fund_similar_mean_pd, 0.3)
fund_similar_top_pd.to_csv("./fund_similar_top_per_" + str(0.5) +"y_pd.csv", index=False, encoding='utf_8_sig')

In [19]:
# 统计5 年来同类排名数平均值前 30%的基金
for i in range(1,6,1):
    fund_similar_mean_pd = get_similar_mean(i)
    fund_similar_top_pd = get_similar_top(fund_similar_mean_pd, 0.3)
    fund_similar_top_pd.to_csv("./fund_similar_top_" + str(i) +"y_pd.csv", index=False, encoding='utf_8_sig')
# print(fund_similar_mean_pd)
# 半年
fund_similar_mean_pd = get_similar_mean(0.5)
fund_similar_top_pd = get_similar_top(fund_similar_mean_pd, 0.3)
fund_similar_top_pd.to_csv("./fund_similar_top_" + str(0.5) +"y_pd.csv", index=False, encoding='utf_8_sig')

In [20]:
# 找出 5 年同类百分比平均值都处于前 30% 的基金
fund_similar_top_6m_pd = pd.read_csv('./fund_similar_top_per_0.5y_pd.csv',low_memory=False, header=None)
fund_similar_top_1y_pd = pd.read_csv('./fund_similar_top_per_1y_pd.csv',low_memory=False, header=None)
fund_similar_top_2y_pd = pd.read_csv('./fund_similar_top_per_2y_pd.csv',low_memory=False, header=None)
fund_similar_top_3y_pd = pd.read_csv('./fund_similar_top_per_3y_pd.csv',low_memory=False, header=None)
fund_similar_top_4y_pd = pd.read_csv('./fund_similar_top_per_4y_pd.csv',low_memory=False, header=None)
fund_similar_top_5y_pd = pd.read_csv('./fund_similar_top_per_5y_pd.csv',low_memory=False, header=None)
# print(fund_similar_top_1y_pd.shape)
# print(fund_similar_top_2y_pd.shape)
m1 = pd.merge(fund_similar_top_6m_pd,fund_similar_top_1y_pd,how='inner',on = 0)
m2 = pd.merge(m1,fund_similar_top_2y_pd,how='inner',on = 0)
m3 = pd.merge(m2,fund_similar_top_3y_pd,how='inner',on = 0)
m4 = pd.merge(m3,fund_similar_top_4y_pd,how='inner',on = 0)
m5 = pd.merge(m4,fund_similar_top_5y_pd,how='inner',on = 0)
m5.to_csv("./fund_similar_top_per_6m-5y_pd.csv", encoding='utf_8_sig', index=False, header=False)
print(m5.shape)

(56, 7)


In [21]:
# 找出 5 年同类排名数平均值都处于前 30% 的基金
fund_similar_top_6m_pd = pd.read_csv('./fund_similar_top_0.5y_pd.csv',low_memory=False, header=None)
fund_similar_top_1y_pd = pd.read_csv('./fund_similar_top_1y_pd.csv',low_memory=False, header=None)
fund_similar_top_2y_pd = pd.read_csv('./fund_similar_top_2y_pd.csv',low_memory=False, header=None)
fund_similar_top_3y_pd = pd.read_csv('./fund_similar_top_3y_pd.csv',low_memory=False, header=None)
fund_similar_top_4y_pd = pd.read_csv('./fund_similar_top_4y_pd.csv',low_memory=False, header=None)
fund_similar_top_5y_pd = pd.read_csv('./fund_similar_top_5y_pd.csv',low_memory=False, header=None)
# print(fund_similar_top_1y_pd.shape)
# print(fund_similar_top_2y_pd.shape)
m1 = pd.merge(fund_similar_top_6m_pd,fund_similar_top_1y_pd,how='inner',on = 0)
m2 = pd.merge(m1,fund_similar_top_2y_pd,how='inner',on = 0)
m3 = pd.merge(m2,fund_similar_top_3y_pd,how='inner',on = 0)
m4 = pd.merge(m3,fund_similar_top_4y_pd,how='inner',on = 0)
m5 = pd.merge(m4,fund_similar_top_5y_pd,how='inner',on = 0)
m5.to_csv("./fund_similar_top_6m-5y_pd.csv", encoding='utf_8_sig', index=False, header=False)
print(m5.shape)

(124, 7)


In [22]:
# 获取 code 对应的类型
def get_fund_type_name(code):
    try:
        type_name = fund_wide_pd_type.loc[fund_wide_pd_type['code'] == code]['类型']
    #     print(len(type_name))
        if(len(type_name) > 0):
            return type_name.values[0]
        else:
            return ""
    except:
        return "111"

In [23]:
# 同时满足五年内同类排名数和同类排名百分比都 30%的基金
fund_similar_top_per_6m_5y_pd = pd.read_csv("./fund_similar_top_per_6m-5y_pd.csv", encoding='utf_8_sig',low_memory=False)
fund_similar_top_6m_5y_pd = pd.read_csv("./fund_similar_top_6m-5y_pd.csv", encoding='utf_8_sig',low_memory=False)
similar_top_all = pd.merge(fund_similar_top_per_6m_5y_pd,fund_similar_top_6m_5y_pd,how='inner',on = 'code')
similar_top_all.to_csv("./similar_top_all_6m-5y_pd.csv", encoding='utf_8_sig')
print(similar_top_all.shape)

(30, 13)


In [24]:
# 读取相同类型排名书 top 30% 的文件并添加类型
fund_similar_top_6m_5y_pd = pd.read_csv("./similar_top_all_6m-5y_pd.csv", encoding='utf_8_sig')
# 创建空列，dtype = object 可以存多个字符
type_name_np = np.empty(fund_similar_top_6m_5y_pd.shape[0], dtype = object)
# 遍历 df
for i in range(0, len(fund_similar_top_6m_5y_pd)):
# for i in range(0, 3):
    code = fund_similar_top_6m_5y_pd.iloc[i]['code'].astype(int) # 获取 code
    type_name = get_fund_type_name(str(code).zfill(6)) # 获取 类型
    type_name_np[i] = type_name # 保存名称
# 插入type_name列
fund_similar_top_6m_5y_pd.insert(1,'type_name', type_name_np)
# 保存文件
fund_similar_top_6m_5y_pd.to_csv("./similar_top_all_6m-5y_pd_type_name.csv", encoding='utf_8_sig',index=False)


In [258]:
# get_recent_year_mean(4,'净值回报',worth_pd)

空经理 code 001729


(6680, 49)
(6680, 32)
(6377, 18)


In [346]:
print(fund_wide_pd.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7623 entries, 0 to 7622
Data columns (total 32 columns):
name           7623 non-null object
code           7623 non-null object
类型             7623 non-null object
原费率            5857 non-null object
现费率            5851 non-null object
最小申购金额         5853 non-null object
近一年收益率         5911 non-null object
近6月收益率         6575 non-null object
近三月收益率         6991 non-null object
近一月收益率         7231 non-null object
基金持仓股票代码       7622 non-null object
基金持仓债券代码       7623 non-null object
规模变动日期         7623 non-null object
规模变动净资产亿元      7623 non-null object
规模变动较上期环比      7623 non-null object
('机构持有比例',)    7623 non-null object
('个人持有比例',)    7623 non-null object
('内部持有比例',)    7623 non-null object
('股票占净比',)     7623 non-null object
('债券占净比',)     7623 non-null object
('现金占净比',)     7623 non-null object
('净资产',)       7623 non-null object
('平均值',)       7623 non-null object
('选证能力',)      4772 non-null object
('收益率',)       4772 non-null 

In [ ]:
# 测试代码分割线

In [35]:
allCode = getAllCode()
# allCode = ['040001','003150']
for code in allCode:
    fund_data = grandTotal_pd.loc[grandTotal_pd[1] == int(code)]
#     print("fund_data",fund_data.shape)
    if fund_data.shape[0] > 0:
            
        fund_data1 = fund_data.loc[fund_data[0] == '同类平均累计收益率']
#         print("fund_data1",fund_data1.shape)
        if fund_data1.shape[0] == 0:
            print(code)
            


150008
150009
150012
150013
150018
150019
150022
150023
150028
150029
150030
150031
150051
150052
150053
150054
150055
150056
150059
150060
150064
150065
150083
150084
150092
150093
150094
150095
150100
150101
150104
150105
150106
150107
150117
150118
150123
150124
150130
150131
150135
150136
150143
150144
150145
150146
150148
150149
150150
150151
150152
150153
150157
150158
150164
150165
150167
150168
150171
150172
150173
150174
150177
150178
150179
150180
150181
150182
150184
150185
150186
150187
150190
150191
150192
150193
150194
150195
150196
150197
150198
150199
150200
150201
150203
150204
150205
150206
150207
150208
150209
150210
150211
150212
150213
150214
150215
150216
150217
150218
150219
150220
150221
150222
150223
150224
150225
150226
150227
150228
150229
150230
150231
150232
150235
150236
150237
150238
150243
150244
150245
150246
150247
150248
150249
150250
150251
150252
150255
150256
150257
150258
150259
150260
150261
150262
150263
150264
150265
150266
150267
150268
150269

In [127]:
a = get_recent_manage_info('000086')
a.insert(0,'code','003150')
# fund_wide_pd.loc[fund_wide_pd['code'] == code]
# print(a)
# b = get_recent_manage_info('003150')
# b.insert(0,'code','003150')

# a = a.append(b, ignore_index=True)
print(a)
# 003150 需要重新爬、已解决

     code      经理ID  经理星级 经理在这个基金工作时间  经理资金数 经理基金数 经理平均分 经理经验值 经理收益率 经理抗风险 经理稳定性 经理择时能力   经理任期收益 经理同类平均 经理沪深300 经理跟踪误差 经理超额收益 经理管理规模
0  003150  30289498     0        1881  23.52     4  暂无数据  None  None  None  None      0  16.5779  20.52       0      0      0   None


In [500]:
# a1 = fund_wide_pd.loc[fund_wide_pd['code'] == '000184']['经理'].values[i].split( )
# print(len(a1))
# print(a1)
# print(fund_wide_pd.loc[fund_wide_pd['code'] == '000184']['经理'].values[i])

# a3 = fund_wide_pd.loc[fund_wide_pd['code'] == '000184']['经理'].values[i].split('\t')
a1 = fund_wide_pd.loc[fund_wide_pd['code'] == '000184']['经理'].values[0].split('\n')
# print(len(a1))
# 
col_num = len(a1[0].split()) + 1
print(col_num)
manage_pd = pd.DataFrame()
a1[0]  = '列名 ' + a1[0]
# aaa= cname.append()
# print(aaa)
# 把经理恢复成 pd，可操作
for row in a1:
    row_s = row.split()
    series = pd.Series(row_s[1:],name=row_s[0]).T
    manage_pd = manage_pd.append(series)
    
#     manage_pd = manage_pd.append(a,name="a")
    
print(manage_pd)
#     b = np.array([b, r])
#     print()
#     print(len(r.split()))
# a2 = a1.split('\n')

# print(len(a3))
# print(a3)
# aaa = pd.DataFrame(fund_wide_pd.loc[fund_wide_pd['code'] == '000184']['经理'].values[i].split( ))
# bbb = np.array(fund_wide_pd.loc[fund_wide_pd['code'] == '000184']['经理'].values[i].split( ))
# ccc = bbb.reshape[:,2]
# print(bbb.shape)
# print(bbb)
# print(manages_list[4])

3
                      0              1
列名                    0              1
id             30364517       30568351
姓名                   张洋             李昱
星级                    4              4
工作时间             4年又27天        1年又232天
资金/基金数    44.10亿(16只基金)  18.70亿(11只基金)
平均分               64.57          60.63
截止日期         2019-09-11     2019-09-11
(经验值,)             60.3           28.4
(收益率,)             22.9           34.7
(抗风险,)             97.8           87.3
(稳定性,)             98.3           91.3
(择时能力,)            99.8           85.6
(任期收益,)         -1.0348        -4.4132
(同类平均,)            6.42           8.09
(沪深300,)          15.59         -10.33


In [338]:
# mean = fund_similar_mean_pd.iloc[0,:].mean(0)
# print(fund_similar_mean_pd)
fund_similar_mean_pd_t = fund_similar_mean_pd
fund_similar_mean_pd_t.sort_values(by = '同类平均值', inplace=True, ascending=False)
pd_len = fund_similar_mean_pd_t.shape[0]
print(pd_len * 0.3)

print(fund_similar_mean_pd_t.shape)
# fund_similar_mean_pd_t = fund_similar_mean_pd_t.iloc[0:round(pd_len * 0.3)]
fund_similar_mean_pd_t = fund_similar_mean_pd_t.head(round(pd_len * 0.3))
print(fund_similar_mean_pd_t.shape)
# round(pd_len * 0.3)
# .head(3)


251.7
(839, 2)
(252, 2)


In [119]:
fund_similar = rateInSimilar_pd.loc[(rateInSimilar_pd[0] == "同类排名百分比")
                                  & (rateInSimilar_pd[1] == 100022)]
# fund_similar.drop([0, 1, 2], axis=1, inplace=True)  # 删除前三列无用数据
fund_similar = fund_similar.T
# fund_similar = fund_similar.iloc[2:,:]
print(fund_similar)
fund_similar = fund_similar[fund_similar.iloc[:, 0] > 0]  # 只要有效数据
#         还需要找 x 天内的数据
# print(fund_similar)
# print(fund_similar.iloc[:,5*365].shape)
# fund_similar = fund_similar.iloc[5*365,:]
print(fund_similar.iloc[0:5*365,:].shape)
mean = fund_similar.mean(0).iloc[0]  # 求平均
print(mean.shape)
print(mean)
print(type(mean))

        22711
0     同类排名百分比
1      100022
2        74.5
3       75.32
4          79
5       79.68
6       81.57
7        81.6
8       82.19
9       82.47
10      80.34
11       81.1
12      80.62
13      79.99
14      82.29
15      83.11
16      74.74
17      73.45
18      71.62
19       70.6
20      66.37
21      63.83
22      72.37
23      69.59
24      66.45
25      71.47
26      61.11
27      57.23
28      66.52
29      33.52
...       ...
4273        0
4274        0
4275        0
4276        0
4277        0
4278        0
4279        0
4280        0
4281        0
4282        0
4283        0
4284        0
4285        0
4286        0
4287        0
4288        0
4289        0
4290        0
4291        0
4292        0
4293        0
4294        0
4295        0
4296        0
4297        0
4298        0
4299        0
4300        0
4301        0
4302        0

[4303 rows x 1 columns]
(1825, 1)


IndexError: single positional indexer is out-of-bounds

In [239]:

fund_day = fund_narrow_pd.loc[(fund_narrow_pd[0] == "时间戳") & (fund_narrow_pd[1] ==  int("040023"))]
fund_day = fund_day.T
print("fund_day", fund_day.shape)

x1 = fund_day[fund_day.iloc[:,0] != 0]
print(x1.shape)
# print(x1)




fund_day (4344, 1)
(2002, 1)


Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]
Index: []

[0 rows x 4344 columns]
